# **Homework 2**
**Name:** Amirhossein Jamali

**GitHub repository:** https://github.com/jamir91/NLP

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bidi.algorithm import get_display
from arabic_reshaper import reshape

In [17]:
class DataPreprocessor:
    train_data = pd.DataFrame()
    test_data = pd.DataFrame()
    cleaned_data = None

    def __init__(self, address='./data/'):
        self.read_data(address)
        self.plot_distribution()

    def read_data(self, address):
        self.train_data = pd.read_csv(address + 'train.csv', sep='\t', error_bad_lines= False , encoding= 'utf-8')
        self.train_data.drop(self.train_data[self.train_data.category=='category'].index, axis=0, inplace=True)
        self.train_data.dropna(subset=['text'], inplace=True)
        self.test_data = pd.read_csv(address + 'test.csv', sep='\t', error_bad_lines= False , encoding= 'utf-8')
        self.test_data.drop(self.test_data[self.test_data.category=='category'].index, axis=0, inplace=True)
        self.test_data.dropna(subset=['text'], inplace=True)
#         print(self.train_data.info())
#         print('-------------------------------------------------------------')
#         print(self.test_data.info())

    def draw_plot(self, labels, counts, fname):
        font = {"family": "B Nazanin", "size": 15} 
        plt.rc("font", **font)

        persian_labels = [get_display(reshape(label)) for label in labels]
        fig = plt.figure(figsize = (10, 5))
        plt.bar(persian_labels, counts, width = 0.7, color='darkblue')

        for index,data in enumerate(counts):
            plt.text(x=index, y =data+1, s=f"{data}", color='darkgreen', fontdict=dict(fontsize=13))

        plt.savefig(fname, dpi=200)

    def plot_distribution(self):
        labels, counts = np.unique(self.train_data['category'], return_counts=True)
        self.draw_plot(labels, counts, 'train.png')
        print('-------------------------------------------------------------')
        labels, counts = np.unique(self.test_data['category'], return_counts=True)
        self.draw_plot(labels, counts, 'test.png')
    
    def replace_digits(d):
        persian_digits = u'0123456789۱۲۳۴۵۶۷۸۹۰'
        for c in d:
            if c in persian_digits:
                d = d.replace(c, 'N')
        while 'NN' in d:
            d = d.replace('NN', 'N')
        while 'N.N' in d:
            d = d.replace('N.N', 'N')
        return d

    def clean_text(self):
        pass

    def count_words(self):
        pass

    def mao_word_index(self):
        pass

    def tokenize(self):
        pass

In [30]:
import pandas as pd

train_df = pd.read_csv('./data/train.csv', sep='\t', error_bad_lines= False , encoding= 'utf-8')
test_df = pd.read_csv('./data/test.csv', sep='\t', error_bad_lines= False , encoding= 'utf-8')

print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117192 entries, 0 to 117191
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   link       117192 non-null  object
 1   title      117161 non-null  object
 2   code_news  117161 non-null  object
 3   category   117161 non-null  object
 4   date       117161 non-null  object
 5   text       97030 non-null   object
dtypes: object(6)
memory usage: 5.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21104 entries, 0 to 21103
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   link       21104 non-null  object
 1   title      21099 non-null  object
 2   code_news  21099 non-null  object
 3   category   21099 non-null  object
 4   date       21099 non-null  object
 5   text       17502 non-null  object
dtypes: object(6)
memory usage: 989.4+ KB
None


In [31]:
train_df[train_df.category=='category']

,link,title,code_news,category,date,text
12561,link,title,code_news,category,date,text
12847,link,title,code_news,category,date,text
14984,link,title,code_news,category,date,text
28402,link,title,code_news,category,date,text
53403,link,title,code_news,category,date,text
67967,link,title,code_news,category,date,text
80702,link,title,code_news,category,date,text
96991,link,title,code_news,category,date,text


In [32]:
train_df.drop(train_df[train_df.category=='category'].index, axis=0, inplace=True)
train_df.dropna(subset=['text'], inplace=True)

In [33]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97022 entries, 0 to 117191
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   link       97022 non-null  object
 1   title      97022 non-null  object
 2   code_news  97022 non-null  object
 3   category   97022 non-null  object
 4   date       97022 non-null  object
 5   text       97022 non-null  object
dtypes: object(6)
memory usage: 5.2+ MB


In [34]:
from __future__ import unicode_literals
from hazm import Normalizer, Stemmer, Lemmatizer, word_tokenize

In [35]:
def replace_digits(d):
    persian_digits = u'0123456789۱۲۳۴۵۶۷۸۹۰'
    for c in d:
        if c in persian_digits:
            d = d.replace(c, 'N')
    while 'NN' in d:
        d = d.replace('NN', 'N')
    while 'N.N' in d:
        d = d.replace('N.N', 'N')
    return d

In [37]:
train_df['text'][1]

' به گزارش حوزه موسیقی باشگاه خبرنگاران، مهدی یغمایی، خواننده\u200cی جوان و خوش ذوق موسیقی پاپ به گفته خود یکی از نوادگان "یغمای جندقی" شاعر مرثیه سرای قرن 13 هجری است، وی در سال 89 در آلبوم زیبا و دلنشین "سلام آقا" که به پیشگاه مقدس و سرور و سالار شهیدان و خاندان پاکشان تقدیم شد، به همراه جمعی از ستارگان موسیقی پاپ به اجرای قطعات ماندگار و دلنشین پرداخت که به این بهانه به گفتگو با وی نشسته\u200cایم. *"آقای یغمایی" در ساخت قطعاتی که به پیشگاه مقدس امام حسین (ع) و خاندان پاک ایشان تقدیم می\u200cشود، چه عواملی باید وجود داشته باشد؟ *"آقای یغمایی" در ساخت قطعاتی که به پیشگاه مقدس امام حسین (ع) و خاندان پاک ایشان تقدیم می\u200cشود، چه عواملی باید وجود داشته باشد؟ ساخت این قطعات، مستلزم اعتقاد قلبی عمیق است در آلبوم "سلام آقا" من بدون هیچ قراردادی 4 قطعه را اجرا کردم و به طور کلی از عمق وجودم به اجرای این قطعات پرداختم.در این مجموعه برای من انگیزه مالی به هیچ عنوان مطرح نبوده و وقتی این آلبوم مجوز گرفت، سجده شکر کردم. ساخت این قطعات، مستلزم اعتقاد قلبی عمیق است در آلبوم "سلام آقا" من بدون ه

In [39]:
train_df['text'] = train_df['text'].apply(normalizer.normalize)
train_df['text'] = train_df['text'].apply(replace_digits)
train_df['text'][1]

' به گزارش حوزه موسیقی باشگاه خبرنگاران، مهدی یغمایی، خواننده\u200cی جوان و خوش ذوق موسیقی پاپ به گفته خود یکی از نوادگان «یغمای جندقی» شاعر مرثیه سرای قرن N هجری است، وی در سال N در آلبوم زیبا و دلنشین «سلام آقا» که به پیشگاه مقدس و سرور و سالار شهیدان و خاندان پاکشان تقدیم شد، به همراه جمعی از ستارگان موسیقی پاپ به اجرای قطعات ماندگار و دلنشین پرداخت که به این بهانه به گفتگو با وی نشسته\u200cایم. * «آقای یغمایی» در ساخت قطعاتی که به پیشگاه مقدس امام حسین (ع) و خاندان پاک ایشان تقدیم می\u200cشود، چه عواملی باید وجود داشته باشد؟ * «آقای یغمایی» در ساخت قطعاتی که به پیشگاه مقدس امام حسین (ع) و خاندان پاک ایشان تقدیم می\u200cشود، چه عواملی باید وجود داشته باشد؟ ساخت این قطعات، مستلزم اعتقاد قلبی عمیق است در آلبوم «سلام آقا» من بدون هیچ قراردادی N قطعه را اجرا کردم و به طور کلی از عمق وجودم به اجرای این قطعات پرداختم. در این مجموعه برای من انگیزه مالی به هیچ عنوان مطرح نبوده و وقتی این آلبوم مجوز گرفت، سجده شکر کردم. ساخت این قطعات، مستلزم اعتقاد قلبی عمیق است در آلبوم «سلام آقا» من بدون 

In [27]:
normalizer = Normalizer()

In [29]:
normalizer.normalize('اصلاح نويسه ها و استفاده از نیم‌فاصله پردازش را آسان مي كند')

'اصلاح نوییسه\u200cها و استفاده از نیم\u200cفاصله پردازش را آسان می\u200cکند'

In [ ]:
def sent_tokenizer(text):
    # sentence separators: ؟ . ! ?
    seperators = "؟.!?"
    pattern = r"[^{0}]+[{0}]".format(seperators)
    return re.findall(pattern, text)

In [ ]:
def word_tokenizer(sent):
    seperators = "؟!.?,:;، \t"
    pattern = r"([^{0}]+)([{0}])".format(seperators)

    words_sep = re.findall(pattern, sent)
    words = [w for word, sep in words_sep for w in [word, sep] if w != " "]
    return words

In [ ]:
import re
tokenized = [word_tokenizer(sent) for sent in sent_tokenizer(train_df['text'][200])]
tokenized[3]

In [ ]:
! pip install hazm

In [ ]:
from __future__ import unicode_literals
from hazm import Normalizer, Stemmer, Lemmatizer, word_tokenize

In [ ]:
word_tokenizer(sent) for sent in sent_tokenizer(train_df['text'][200])

In [ ]:
normalizer = Normalizer()
tokenized = normalizer.normalize(train_df['text'][200])
print(tokenized)
tokenized = word_tokenize(tokenized)
print(tokenized)
stemmer = Stemmer()
for i in tokenized:
    print(stemmer.stem(i))
    
# tokenized = stemmer.stem(train_df['text'][200])

print(stemmer.stem('دستان'))

# print(tokenized)

In [ ]:
if __name__ == '__main__':
    dp = DataPreprocessor()